In [ ]:
import json
import requests
import pandas as pd
from datasets import load_dataset

In [ ]:
class FFMLLama2:
    def __init__(self, api_key, model_name, api_url, max_new_tokens=350, temperature=0.5, top_k=50, top_p=1.0, frequency_penalty=1.0):
        self.api_key = api_key
        self.model_name = model_name
        self.api_url = api_url
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature
        self.top_k = top_k
        self.top_p = top_p
        self.frequency_penalty = frequency_penalty

    def conversation(self, system, contents):
        headers = {
            "content-type": "application/json",
            "X-API-Key": self.api_key}
        roles  = ["human", "assistant"]
        messages = []
        if system is not None:
            messages.append({"role":"system", "content": system})
        for index, content in enumerate(contents):
            messages.append({"role": roles[index%2], "content": content})
        data = {
            "model":self.model_name,
            "message":messages,
            "parameters": {
                "max_new_tokens": self.max_new_tokens,
                "temperature": self.temperature,
                "top_k": self.top_k,
                "top_p": self.top_p,
                "frequency_penalty": self.frequency_penalty
                }
        }
        print(messages)

        result = ""
        response = requests.post(self.api_url + "/api/models/conversation", json=data, headers=headers, verify=False)
        print(response.status_code)
        if response.status_code == 200:
            print(response)
            result = json.loads(response.text, strict=False)['generated_text']
            with open('answer_record.json', 'a', encoding='utf-8-sig') as f:
                    json.dump(result, f, ensure_ascii=False, indent=2)
        else:
             print(f"error, status_code:{response.status_code}")
        return result.strip("\n")

In [ ]:
MODEL_NAME = "ffm-llama2-70b-chat-exp"
API_KEY = "98430d42-c60c-4612-adbb-eeb927c5c2be"
API_URL = "https://53955.afs.twcc.ai/text-generation"

In [ ]:
ffmllama =FFMLLama2(API_KEY, MODEL_NAME, API_URL)
system_prompt = "You are an AI assistant, Please read the following documents than choose an answer from A, B, C, D."
contents = ["關於腦性麻痺的敘述，下列何者錯誤? A_牙科治療要使用張口器前須慎重考慮，以免刺激病人的嘔吐反射, B_痙攣性腦性麻痺佔腦性麻痺患者的約7成, C_paraplegia腦性麻痺患者是指單側肢體受影響的腦性麻痺, D_治療過程盡量為維持病人頭部穩定，四肢盡量靠近軀幹"] 
result = ffmllama.conversation(system_prompt, contents)

In [ ]:
print(result)